<a href="https://colab.research.google.com/github/vineetyadav/DataScienceR/blob/master/Clinical_trails.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
##Clinical Patient Notes Kagggle 


# New Section

# Loading DataSet in google colab 

Install Kaggle Library 

In [1]:
! pip install kaggle


 Create a directory named “.kaggle”

In [2]:
! mkdir ~/.kaggle

Copy Kaggle Json in directory 

In [3]:
! cp kaggle.json ~/.kaggle/


Provide permission to File 

In [4]:
! chmod 600 ~/.kaggle/kaggle.json

Downloading Competitions dataset:

In [5]:
! kaggle competitions download -c nbme-score-clinical-patient-notes

 61% 6.00M/9.83M [00:00<00:00, 60.9MB/s]
100% 9.83M/9.83M [00:00<00:00, 86.6MB/s]


In [6]:
!ls


kaggle.json  nbme-score-clinical-patient-notes.zip  sample_data


In [7]:
!unzip /content/nbme-score-clinical-patient-notes.zip 

Archive:  /content/nbme-score-clinical-patient-notes.zip
  inflating: features.csv            
  inflating: patient_notes.csv       
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


# Exploring Dataset

## import all required packages 

In [8]:
import pandas as pd
import matplotlib.pyplot as mlpt 
import numpy as np
import os 


Exploring training, test and original dataset. 

In [9]:
dataset = pd.read_csv("patient_notes.csv")
dataset.head()

,pn_num,case_num,pn_history
0,0,0,"17-year-old male, has come to the student heal..."
1,1,0,17 yo male with recurrent palpitations for the...
2,2,0,Dillon Cleveland is a 17 y.o. male patient wit...
3,3,0,a 17 yo m c/o palpitation started 3 mos ago; \...
4,4,0,17yo male with no pmh here for evaluation of p...


In [10]:
train_dataset = pd.read_csv("train.csv")
train_dataset.head()

,id,case_num,pn_num,feature_num,annotation,location
0,00016_000,0,16,0,['dad with recent heart attcak'],['696 724']
1,00016_001,0,16,1,"['mom with ""thyroid disease']",['668 693']
2,00016_002,0,16,2,['chest pressure'],['203 217']
3,00016_003,0,16,3,"['intermittent episodes', 'episode']","['70 91', '176 183']"
4,00016_004,0,16,4,['felt as if he were going to pass out'],['222 258']


In [11]:
test_dataset = pd.read_csv("test.csv")
test_dataset.head()

,id,case_num,pn_num,feature_num
0,00016_000,0,16,0
1,00016_001,0,16,1
2,00016_002,0,16,2
3,00016_003,0,16,3
4,00016_004,0,16,4


Merge train dataset with orignal dataset 

In [12]:
merged_train_dataset = train_dataset.merge(dataset,on="pn_num",how="inner")
merged_train_dataset.head()

,id,case_num_x,pn_num,feature_num,annotation,location,case_num_y,pn_history
0,00016_000,0,16,0,['dad with recent heart attcak'],['696 724'],0,HPI: 17yo M presents with palpitations. Patien...
1,00016_001,0,16,1,"['mom with ""thyroid disease']",['668 693'],0,HPI: 17yo M presents with palpitations. Patien...
2,00016_002,0,16,2,['chest pressure'],['203 217'],0,HPI: 17yo M presents with palpitations. Patien...
3,00016_003,0,16,3,"['intermittent episodes', 'episode']","['70 91', '176 183']",0,HPI: 17yo M presents with palpitations. Patien...
4,00016_004,0,16,4,['felt as if he were going to pass out'],['222 258'],0,HPI: 17yo M presents with palpitations. Patien...


In [13]:
loc = list(merged_train_dataset.loc[(merged_train_dataset.pn_num==224) & (merged_train_dataset.location!='[]'), 'location'].str.replace("['", "", regex=False).str.replace("']", "", regex=False))


In [14]:
import spacy
from spacy import displacy
nlp = spacy.blank('en')

In [15]:
text = merged_train_dataset[merged_train_dataset.pn_num==224].pn_history.values[0]
doc = nlp.make_doc(text)
ents = []
for l in loc:
    start, end = l.split(' ')
    ent = doc.char_span(int(start), int(end), label='annotation')
    ents.append(ent)
doc.ents = ents
color = {"Annotation": '#A32EFF'}
displacy.render(doc, style="ent", jupyter=True, options={'colors': color})

##Merge Test dataset with orignal dataset 

In [16]:
merged_test_dataset = test_dataset.merge(dataset,on="pn_num",how="inner")
merged_test_dataset.head()

,id,case_num_x,pn_num,feature_num,case_num_y,pn_history
0,00016_000,0,16,0,0,HPI: 17yo M presents with palpitations. Patien...
1,00016_001,0,16,1,0,HPI: 17yo M presents with palpitations. Patien...
2,00016_002,0,16,2,0,HPI: 17yo M presents with palpitations. Patien...
3,00016_003,0,16,3,0,HPI: 17yo M presents with palpitations. Patien...
4,00016_004,0,16,4,0,HPI: 17yo M presents with palpitations. Patien...


In [17]:
merged_train_dataset['location'] = merged_train_dataset['location'].apply(ast.literal_eval)
merged_train_dataset.head()


NameError: ignored

In [18]:
from ast import literal_eval
merged_train_dataset["location_list"] = [literal_eval(x) for x in merged_train_dataset["location"]]

In [19]:
merged_train_dataset["location_new"] = merged_train_dataset["location"].replace("[","",regex=False).replace("]","",regex=False)
merged_train_dataset["location_new"]

0                 ['696 724']
1                 ['668 693']
2                 ['203 217']
3        ['70 91', '176 183']
4                 ['222 258']
                 ...         
14295                      []
14296                      []
14297             ['274 282']
14298             ['421 437']
14299             ['314 330']
Name: location_new, Length: 14300, dtype: object

In [20]:
merged_train_dataset["location_list"]

0               [696 724]
1               [668 693]
2               [203 217]
3        [70 91, 176 183]
4               [222 258]
               ...       
14295                  []
14296                  []
14297           [274 282]
14298           [421 437]
14299           [314 330]
Name: location_list, Length: 14300, dtype: object

In [21]:
! pip install spacy

In [22]:
import spacy.cli
spacy.cli.download("en_core_web_sm")


✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


In [23]:
entity ="Phrase"
import ast
#print(merged_train_dataset.columns)
#merged_train_dataset.apply(lambda x: (, 'cols': x.tolist()[1:]), axis=1).tolist()
x = merged_train_dataset.apply(lambda x: (x['pn_history'],{'entries':[(ast.literal_eval(x['location']),entity)]}),axis=1)
print(x[0][1])



{'entries': [(['696 724'], 'Phrase')]}


In [64]:
import random
from spacy.util import minibatch, compounding
from pathlib import Path
def ner_train(train_instance):
  # Disable pipeline components you dont need to change
  nlp=spacy.load('en_core_web_sm')
  pipe_exceptions = ["ner", "trf_wordpiecer", "trf_tok2vec"]
  unaffected_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]
  
  # Training for 30 iterations
  for iteration in range(30):

    # shuufling examples  before every iteration
    random.shuffle(train_instance)
    losses = {}
    # batch up the examples using spaCy's minibatch
    batches = minibatch(train_instance, size=compounding(4.0, 32.0, 1.001))
    for batch in batches:
        texts, annotations = zip(*batch)
        nlp.update(
                    texts,  # batch of texts
                    annotations,  # batch of annotations
                    drop=0.5,  # dropout - make it harder to memorise data
                    losses=losses,
                )
        print("Losses", losses)
  


In [67]:
def get_train_data(pn_num):
  paragraph_subset = merged_train_dataset[merged_train_dataset["pn_num"]==pn_num]
  locations = paragraph_subset["location"].apply(eval)
  text = paragraph_subset["pn_history"].head(1).item()
  #print(len(text))
  #print("Hi",text)
  ents = []
  entity_set = []
  start_entity = []
  end_entity = [] 
  for location in locations:
        for i in range(len(location)):
            for loc in location:
                val = loc.split()
                val_1 = [int(each) for each in val]
                if "-".join(val) not in entity_set:
                  ents.append(val_1+["Entity"])
                  entity_set.append("-".join(val))
                  entity_set = set(entity_set)
                  entity_set = list(entity_set)

  final_entity_dict = {"entities":ents}
  final_res = (text,final_entity_dict)
  return final_res
get_train_data(46)

('Mr. Cleveland is a 17yo M who was consented by parents to be alone today and provide information.  He has no notable PMH and presented today with palpitations that have been occurring sporadically for 2-3 months.  He started college 7-8 months ago and described feelings of nervousness and anxiousness with starting school and exams.  He endorses using his roommates prescription adderall to study for tests and this coincides with the palpitations at times.  Nothing makes his palpitations worse or better when they occur.  ROS is negative except for light headedness and SOB.  He has not had any cold/heat intolerance, diarrhea, changes in voiding habits, or weight loss.  Worried about being able to play basketball.\r\n\r\nMeds:  Adderall from his roommate\r\nPMH:  none\r\nPSH:  none\r\nAll:  NKDA\r\nFamily:  mom: thyroid issues, father: heart attack\r\nsocial:  no tob, occ EtOH, no illicits\r\nsex:  with gf, use condoms\r\ndiet:  good\r\nexercise:  basketball',
 {'entities': [[824, 844, '

In [68]:

train_data = []
unique_pn_num = merged_train_dataset.pn_num.unique()
for each_pn_num in unique_pn_num:
  try:  
    train_instance = get_train_data(each_pn_num)
    train_data.append(train_instance)
  except:
    print("error for pn_num",each_pn_num )


error for pn_num 82
error for pn_num 211
error for pn_num 225
error for pn_num 234
error for pn_num 344
error for pn_num 372
error for pn_num 390
error for pn_num 464
error for pn_num 495
error for pn_num 510
error for pn_num 575
error for pn_num 583
error for pn_num 653
error for pn_num 669
error for pn_num 673
error for pn_num 699
error for pn_num 701
error for pn_num 719
error for pn_num 768
error for pn_num 777
error for pn_num 796
error for pn_num 804
error for pn_num 810
error for pn_num 817
error for pn_num 821
error for pn_num 831
error for pn_num 861
error for pn_num 883
error for pn_num 970
error for pn_num 987
error for pn_num 1010
error for pn_num 1133
error for pn_num 1146
error for pn_num 1149
error for pn_num 1151
error for pn_num 1165
error for pn_num 1173
error for pn_num 1209
error for pn_num 1250
error for pn_num 1264
error for pn_num 1581
error for pn_num 1587
error for pn_num 1613
error for pn_num 1622
error for pn_num 1694
error for pn_num 1699
error for pn_num 17

In [73]:
#print(train_data[22])
ner_train(train_data)


ValueError: ignored